# Half-twists along the T-R arc

## Modules

In [1]:
import numpy as np
import Permutations as pm
import sympy
from numpy.random import randint as ri
from QuasiCord import *

sympy.init_printing()

import itertools

import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

## Integration

In [134]:
def twist(c, hand):
    t, l, b, r = c[0], c[1], c[2], c[3]
    # ---
    delta = (t + l) - (b + r)
    alpha = 2 * t - delta  # = min(2*t-delta, 2*r+1)
    y = (t - alpha) + (r - alpha)

    f = t - r
    F = abs(2 * f - 1)
    # ---
    position = None
    td, rd = 0, 0

    # -- 1 --
    if (t - alpha >= 0) and (r - alpha >= 0):
        position = "1"
        if (t - alpha == 0) and (r - alpha == 0):
            position += "1"
            if hand == "left":
                td, rd = 1, 0
            else:
                td, rd = 1, 1
        else:
            position += "2"
            if hand == "left":
                td, rd = 2 * y + 1, 2 * (t - alpha)
            else:
                td, rd = 2 * (r - alpha) + 1, 2 * y + 1
    # -- 2 --
    elif (t - alpha) * (r - alpha) <= 0:
        position = "2"
        if r - alpha >= 0:  # --> (t - alpha < 0):
            position += "1"
            if hand == "left":
                td, rd = 2 * (r - alpha) + 1, 2 * (t - alpha)
            else:
                td, rd = F, F
        else:  # r - alpha < 0  (--> t-alpha >=0)
            position += "2"
            if hand == "left":
                td, rd = F, F
            else:
                if 2 * r + 1 >= alpha:
                    position += "1"
                    td, rd = 2 * (r - alpha) + 1, 2 * (t - alpha)
                else:
                    position += "2"
                    td, rd = -2 * r - 1, 2 * (t - 2 * r - 1)
    # -- 3 --
    elif (t - alpha < 0) and (r - alpha < 0):
        position = "3"
        if f > 0:
            position += "1"
            if alpha < 2 * r + 1:  # <--> alpha-t < t-2f-1
                position += "1"
                if hand == "left":
                    td, rd = F, F
                else:
                    if f > alpha - t:
                        position += "1"
                        td, rd = -F, 2 * (t - alpha)
                    else:
                        position += "2"
                        td, rd = -F, -F
            else:
                position += "2"
                if hand == "left":
                    td, rd = F, F
                else:
                    if 2 * r + 1 <= t:
                        position += "1"
                        td, rd = -2 * r - 1, 2 * (t - 2 * r - 1)
                    else:
                        position += "2"
                        if 2 * r + 1 - t < f:
                            position += "1"
                            td, rd = -F, 2 * (t - 2 * r - 1)
                        else:
                            position += "2"
                            td, rd = -F, -F
        else:  # f <= 0
            position += "2"
            if alpha < 2 * r + 1:  # <--> alpha < 2r+1  <--> 2f-1 < delta
                if hand == "left":
                    if alpha - r - 1 >= -f:
                        position += "1"
                        td, rd = -F, -F
                    else:
                        position += "2"
                        td, rd = 2 * (r - alpha) + 1, -F
                else:
                    td, rd = F, F
            else:
                position += "2"
                if hand == "left":
                    td, rd = 0, 0
                else:
                    td, rd = 0, 0
                print(c, position)
    # -- Exception --
    else:
        print(c, alpha, (t - alpha, r - alpha), hand, "Why??")
    # ---
    try:
        sqr = Square(c)
        # ---
        cn = (t + td, l, b, r + rd)
        nsqr = Square(cn)
        # ---
        n = cc(sqr)
        nn = cc(nsqr)
        if n != nn:
            print(
                f"{c}:{n} -- {hand}_{position},{alpha},{t-alpha},{r-alpha} --> {cn}:{nn}"
            )
        # ---
    except Exception as ex:
        print(f"{c} -- {hand}_{position},{alpha},{t-alpha},{r-alpha} --> {cn},{ex}")
        nsqr = sqr
    # ---
    return nsqr, hand + "_" + position

In [154]:
N = 10
Max = 30
# ---
count = 0
while count < Max:
    sqr = square_random_generator(N)
    c = sqr.code
    delta = (c[0] + c[1]) - (c[2] + c[3])
    # ---
    n = cc(sqr)
    sqrnl, posil = twist(c, "left")
    sqrnr, posir = twist(c, "right")
    nnl = cc(sqrnl)
    nnr = cc(sqrnr)
    # Output exceptions
#     if n != nnl:
#         print(f"{n} --{posil}--> {nnl}, {c}")
#     elif n != nnr:
#         print(f"{n} --{posil}--> {nnr}, {c}")
#     else:
#         pass
    #     if (c[0] - delta == 0) and (c[1]-delta)<0:
    #         print(f"{c} --(ltw)--> {twist(c, 'left').code}")
    #     if (c[1] - delta == 0) and (c[0] > c[1]):
    #         print(f"{c,n} --(ltw, {posil})--> {sqrnl.code,nnl}")
    #         print(f"{c,n} --(rtw, {posir})--> {sqrnr.code,nnr}")
    count += 1

(9, 4, 8, 3) -- right_3121,16,-7,-13 --> (2, 4, 8, 7),T+L < B+R
(8, 4, 8, 1) -- right_3121,13,-5,-12 --> (5, 4, 8, 11),T+L < B+R
(8, 2, 7, 2) -- right_3121,15,-7,-13 --> (3, 2, 7, 8),T+L < B+R
(7, 7, 5, 1) -- right_222,6,1,-5 --> (4, 7, 5, 9),T+L < B+R
(6, 3, 1, 2) -- right_222,6,0,-4 --> (1, 3, 1, 4),T+L < B+R
(6, 4, 3, 3):1 -- right_31221,8,-2,-5 --> (1, 4, 3, 1):2
(6, 6, 4, 1) -- right_222,5,1,-4 --> (3, 6, 4, 7),T+L < B+R
(6, 7, 5, 1) -- right_222,5,1,-4 --> (3, 7, 5, 7),T+L < B+R
(7, 4, 8, 2) -- right_3121,13,-6,-11 --> (2, 4, 8, 6),T+L < B+R
(4, 7, 8, 1) -- right_3121,6,-2,-5 --> (1, 7, 8, 3),T+L < B+R


## Are the left and the right inverses of each other?

In [38]:
N = 20
Max = 20
# ---
count, error = 0, 0
while count < Max*50 and error < Max:
    count += 1
    sqr = square_random_generator(N)
    c = sqr.code
    n = cc(sqr)

    sqr_tl, pos_tl = twist(c, "left")
    c_tl = sqr_tl.code
    n_tl = cc(sqr_tl)

    sqr_tl_tr, pos_tl_tr = twist(c_tl, "right")
    c_tl_tr = sqr_tl_tr.code
    n_tl_tr = cc(sqr_tl_tr)

    if c != c_tl_tr:  #pos_tl == 'l1':  #
        error += 1
        print(f"{c}:{n} --[{pos_tl}]-->{c_tl}:{n_tl} --[{pos_tl_tr}]-->{c_tl_tr}:{n_tl_tr}")

In [39]:
N = 10
Max = 20
# ---
count, error = 0, 0
while count < Max*50 and error < Max:
    count += 1
    sqr = square_random_generator(N)
    c = sqr.code
    n = cc(sqr)

    sqr_tr, pos_tr = twist(c, "right")
    c_tr = sqr_tr.code
    n_tr = cc(sqr_tr)

    sqr_tr_tl, pos_tr_tl = twist(c_tr, "left")
    c_tr_tl = sqr_tr_tl.code
    n_tr_tl = cc(sqr_tr_tl)

    if c != c_tr_tl:
        error += 1
        print(f"{c}:{n} --[{pos_tr}]-->{c_tr}:{n_tr} --[{pos_tr_tl}]-->{c_tr_tl}:{n_tr_tl}")

(7, 3, 5, 1) -- right_3121,10,-3,-9 --> (4, 3, 5, 9),T+L < B+R


UnboundLocalError: local variable 'nsqr' referenced before assignment